# FDMNES XAS simulation workflow with Larixite
This notebook shows an example workflow for FDMNES XAS simulation using Larixite.

- Author: Mauro Rovezzi
- Contact: mauro.rovezzi@esrf.fr
- Status: in progress
- Last update: 2025-05-14

First import the necessary modules and set some variables

In [ ]:
%reload_ext autoreload
%autoreload 2

from pathlib import Path
import numpy as np
import crystal_toolkit
import warnings
from larixite.struct import get_structs_from_dir
from larixite.fdmnes import FdmnesXasInput
from larixite.fdmnes import logger

logger.setLevel("INFO")  #: adjust this level if you want to have more or less information: "DEBUG", "INFO", "WARNING", "ERROR", "CRITICAL"
warnings.simplefilter('always', UserWarning)  #: to filter out some technic warnings from underlying pymatgen

#: set a directory containing the structural files (TODO: set your own path)
curdir = Path().cwd()
basedir = curdir.parent
testdir = basedir / "tests"
structsdir = testdir / "structs"

abs = "Zn"  #: select the absorber

fdmnes_executable = "/home/mauro/local/fdmnes_Linux_2024_11_14/fdmnes_Linux/fdmnes_linux64"  #: path to fdmnes executable (TODO: set your own path)

Get a list of XasStructure objects from a directory containing structural files, based on the absorbing element specified before.

In [ ]:
structs = get_structs_from_dir(structsdir, abs, globstr=f"*{abs}*")

select one structure and print some information about it

In [ ]:
sg = structs[0]
sg.show_unique_sites()

the structure can be visualized with the [Crystal Toolkit](https://docs.crystaltoolkit.org/)

In [ ]:
sg.struct

The absorber site is set by default to the first site with the absorbing element, but can be changed by indicating the index in the structure (`istruct` given by the `show_unique_sites()` method) to the `absorber_idx` attribute. It is convenient to check with the `show_unique_sites()` method that the index of the absorbing site is correctly set, as it gets the `(abs)` label next to the index.

In [ ]:
sg.absorber_idx = 0
sg.show_unique_sites()

then generate an FDMNES input file FdmnesXasInput object and show it

In [ ]:
f = FdmnesXasInput(sg, absorber=abs, struct_type="crystal", radius=5)
text = f.get_input()
print(text)

write the FDMNES input file and run FDMNES

In [ ]:
outdir = f.write_input()

In [ ]:
! cd {outdir.as_posix()} && {fdmnes_executable}

The following steps will be showed in the next version of Larixite:

- load the FDMNES output files
- plot the XAS spectrum
- compare the simulated spectrum with experimental data and adjust the convolution parameters
- run multiple jobs for convergence studies